In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime as dt
from matplotlib import pyplot as plt
from collections import Counter, OrderedDict

In [2]:
covid = pd.read_csv("Covid-19 Activity.csv")
covid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 639710 entries, 0 to 639709
Data columns (total 13 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   COUNTY_NAME                      595792 non-null  object 
 1   PEOPLE_POSITIVE_CASES_COUNT      639710 non-null  float64
 2   REPORT_DATE                      639710 non-null  object 
 3   PROVINCE_STATE_NAME              597892 non-null  object 
 4   CONTINENT_NAME                   639710 non-null  object 
 5   DATA_SOURCE_NAME                 639710 non-null  object 
 6   PEOPLE_DEATH_NEW_COUNT           639710 non-null  float64
 7   COUNTY_FIPS_NUMBER               585672 non-null  float64
 8   COUNTRY_ALPHA_3_CODE             639710 non-null  object 
 9   COUNTRY_SHORT_NAME               639710 non-null  object 
 10  COUNTRY_ALPHA_2_CODE             639504 non-null  object 
 11  PEOPLE_POSITIVE_NEW_CASES_COUNT  639710 non-null  float64
 12  PE

In [3]:
covid.head()

,COUNTY_NAME,PEOPLE_POSITIVE_CASES_COUNT,REPORT_DATE,PROVINCE_STATE_NAME,CONTINENT_NAME,DATA_SOURCE_NAME,PEOPLE_DEATH_NEW_COUNT,COUNTY_FIPS_NUMBER,COUNTRY_ALPHA_3_CODE,COUNTRY_SHORT_NAME,COUNTRY_ALPHA_2_CODE,PEOPLE_POSITIVE_NEW_CASES_COUNT,PEOPLE_DEATH_COUNT
0,Wahkiakum,0.0,2020-02-18,Washington,America,New York Times,0.0,53069.0,USA,United States,US,0.0,0.0
1,Wahkiakum,5.0,2020-07-17,Washington,America,New York Times,0.0,53069.0,USA,United States,US,0.0,0.0
2,Wahkiakum,0.0,2020-03-02,Washington,America,New York Times,0.0,53069.0,USA,United States,US,0.0,0.0
3,Stephenson,223.0,2020-06-11,Illinois,America,New York Times,0.0,17177.0,USA,United States,US,1.0,5.0
4,Stephenson,1.0,2020-03-22,Illinois,America,New York Times,0.0,17177.0,USA,United States,US,1.0,0.0


In [4]:
covid.isnull().sum()

COUNTY_NAME                        43918
PEOPLE_POSITIVE_CASES_COUNT            0
REPORT_DATE                            0
PROVINCE_STATE_NAME                41818
CONTINENT_NAME                         0
DATA_SOURCE_NAME                       0
PEOPLE_DEATH_NEW_COUNT                 0
COUNTY_FIPS_NUMBER                 54038
COUNTRY_ALPHA_3_CODE                   0
COUNTRY_SHORT_NAME                     0
COUNTRY_ALPHA_2_CODE                 206
PEOPLE_POSITIVE_NEW_CASES_COUNT        0
PEOPLE_DEATH_COUNT                     0
dtype: int64

In [5]:
# dropping columns that aren't too useful for this analysis
covid.drop(columns=['COUNTRY_ALPHA_2_CODE', 'COUNTY_FIPS_NUMBER'], inplace=True)
covid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 639710 entries, 0 to 639709
Data columns (total 11 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   COUNTY_NAME                      595792 non-null  object 
 1   PEOPLE_POSITIVE_CASES_COUNT      639710 non-null  float64
 2   REPORT_DATE                      639710 non-null  object 
 3   PROVINCE_STATE_NAME              597892 non-null  object 
 4   CONTINENT_NAME                   639710 non-null  object 
 5   DATA_SOURCE_NAME                 639710 non-null  object 
 6   PEOPLE_DEATH_NEW_COUNT           639710 non-null  float64
 7   COUNTRY_ALPHA_3_CODE             639710 non-null  object 
 8   COUNTRY_SHORT_NAME               639710 non-null  object 
 9   PEOPLE_POSITIVE_NEW_CASES_COUNT  639710 non-null  float64
 10  PEOPLE_DEATH_COUNT               639710 non-null  float64
dtypes: float64(4), object(7)
memory usage: 53.7+ MB


In [6]:
# since a count of people will always be a whole number, change floats to ints
covid['PEOPLE_POSITIVE_CASES_COUNT'] = covid['PEOPLE_POSITIVE_CASES_COUNT'].astype(int)
covid['PEOPLE_DEATH_NEW_COUNT'] = covid['PEOPLE_DEATH_NEW_COUNT'].astype(int)
covid['PEOPLE_POSITIVE_NEW_CASES_COUNT'] = covid['PEOPLE_POSITIVE_NEW_CASES_COUNT'].astype(int)
covid['PEOPLE_DEATH_COUNT'] = covid['PEOPLE_DEATH_COUNT'].astype(int)

# filter the rows that hold no information for country name, positive cases, and death counts
no_stats = covid.loc[(covid['COUNTY_NAME'].isnull()) & (covid['PEOPLE_POSITIVE_CASES_COUNT'] == 0) &
         (covid['PEOPLE_DEATH_NEW_COUNT'] == 0) & (covid['PEOPLE_POSITIVE_NEW_CASES_COUNT'] == 0)
         & (covid['PEOPLE_DEATH_COUNT'] == 0)]

# drop those filtered rows and get rid of duplicates
covid.drop(list(no_stats.index), inplace=True)
covid.drop_duplicates(inplace=True)

# for the rest, replace with 'unknown' since they hold stats
covid.fillna('Unknown', inplace=True)
covid.isnull().sum()

COUNTY_NAME                        0
PEOPLE_POSITIVE_CASES_COUNT        0
REPORT_DATE                        0
PROVINCE_STATE_NAME                0
CONTINENT_NAME                     0
DATA_SOURCE_NAME                   0
PEOPLE_DEATH_NEW_COUNT             0
COUNTRY_ALPHA_3_CODE               0
COUNTRY_SHORT_NAME                 0
PEOPLE_POSITIVE_NEW_CASES_COUNT    0
PEOPLE_DEATH_COUNT                 0
dtype: int64

In [7]:
def provincial_case(df, country):
    '''
    Args:
        df: DataFrame
        country: str
    Returns:
        a list of *confirmed* cases of Covid-19 for each province in the country
    Order:
        Descending
    '''
    country_table = df.loc[df['COUNTRY_SHORT_NAME'] == country]
    province_list = country_table['PROVINCE_STATE_NAME'].unique()
    province_dict = dict()
    for province in province_list:
        province_table = country_table.loc[country_table['PROVINCE_STATE_NAME'] == province]
        province_dict[province] = province_table['PEOPLE_POSITIVE_CASES_COUNT'].max()
    return sorted(province_dict.items(), key=lambda x: x[1], reverse=True)


def country_case(df, country):
    '''
    Args:
        df: DataFrame
        country: str
    Returns:
        a sum of *confirmed* cases of Covid-19 in a country
    '''
    province_list = df.loc[df['COUNTRY_SHORT_NAME'] == country]['PROVINCE_STATE_NAME'].unique()
    total = 0
    for province in province_list:
        total += df.loc[df['PROVINCE_STATE_NAME'] == province]['PEOPLE_POSITIVE_NEW_CASES_COUNT'].sum()
    return (country, total)

print(provincial_case(covid, 'Canada'))
print(country_case(covid, 'Canada'))

[('Quebec', 57938), ('Ontario', 38107), ('Alberta', 9861), ('British Columbia', 3362), ('Nova Scotia', 1067), ('Saskatchewan', 1030), ('Manitoba', 361), ('Newfoundland and Labrador', 264), ('New Brunswick', 170), ('Prince Edward Island', 36), ('Repatriated travellers', 13), ('Yukon', 13), ('Northwest Territories', 5)]
('Canada', 112402)


Now that the functions are ready, let's sort countries from the most cases to the least in America

In [8]:
america = covid.loc[covid['CONTINENT_NAME'] == 'America']['COUNTRY_SHORT_NAME'].unique()
america_list = []
for country in america:
    america_list.append(country_case(covid, country))
print(sorted(america_list, key=lambda x:x[1], reverse=True))

[('Uruguay', 11103968), ('Saint Vincent and the Grenadines', 11103968), ('Barbados', 11103968), ('Paraguay', 11103968), ('Bonaire, Saint Eustatius and Saba', 11103968), ('Costa Rica', 11103968), ('Ecuador', 11103968), ('Panama', 11103968), ('El Salvador', 11103968), ('Suriname', 11103968), ('Dominica', 11103968), ('Aruba', 11103968), ('Cayman Islands', 11103968), ('Honduras', 11103968), ('Saint Lucia', 11103968), ('Bahamas', 11103968), ('Grenada', 11103968), ('Trinidad and Tobago', 11103968), ('Haiti', 11103968), ('Turks and Caicos islands', 11103968), ('Cuba', 11103968), ('Antigua and Barbuda', 11103968), ('Venezuela', 11103968), ('Nicaragua', 11103968), ('Sint Maarten', 11103968), ('Belize', 11103968), ('Argentina', 11103968), ('Bolivia', 11103968), ('Chile', 11103968), ('Mexico', 11103968), ('Guatemala', 11103968), ('Colombia', 11103968), ('Dominican Republic', 11103968), ('Guyana', 11103968), ('Bermuda', 11103968), ('Greenland', 11103968), ('Anguilla', 11103968), ('Brazil', 1110396

### Debugging
Almost every country in America got the value of **11103968**

Let's see what's happening.

In [9]:
def country_case(df, country):
    '''
    Args:
        df: DataFrame
        country: str
    Returns:
        a sum of *confirmed* cases of Covid-19 in a country
    '''
    province_list = df.loc[df['COUNTRY_SHORT_NAME'] == country]['PROVINCE_STATE_NAME'].unique()
    total = 0
    print(province_list)

# checking if there's anything in the province column that's causing the problem
print(country_case(covid, 'Uruguay'))
print(country_case(covid, 'Barbados'))
print(country_case(covid, 'Paraguay'))

['Unknown']
None
['Unknown']
None
['Unknown']
None


Found the problem!

The code is summing up cases of all 'Unknown' provinces from all over the world.

To limit the search to within a country, I need to use a temporary dataframe.

In [10]:
def country_case(df, country):
    '''
    Args:
        df: DataFrame
        country: str
    Returns:
        a sum of *confirmed* cases of Covid-19 in a country
    '''
    country_df = df.loc[df['COUNTRY_SHORT_NAME'] == country]
    province_list = country_df['PROVINCE_STATE_NAME'].unique()
    total = 0
    for province in province_list:
        total += country_df.loc[country_df['PROVINCE_STATE_NAME'] == province]['PEOPLE_POSITIVE_NEW_CASES_COUNT'].sum()
    return (country, total)

america = covid.loc[covid['CONTINENT_NAME'] == 'America']['COUNTRY_SHORT_NAME'].unique()
america_list = []
for country in america:
    america_list.append(country_case(covid, country))
countries_continent = sorted(america_list, key=lambda x:x[1], reverse=True)
print(countries_continent)

[('United States', 3980128), ('Brazil', 2227514), ('Peru', 366550), ('Mexico', 362274), ('Chile', 336402), ('Colombia', 218428), ('Argentina', 136105), ('Canada', 112402), ('Ecuador', 77257), ('Bolivia', 64135), ('Dominican Republic', 56043), ('Panama', 55906), ('Guatemala', 41135), ('Honduras', 36102), ('El Salvador', 12975), ('Venezuela', 12774), ('Costa Rica', 12361), ('Haiti', 7167), ('Paraguay', 4000), ('Nicaragua', 3439), ('Cuba', 2462), ('Suriname', 1176), ('Uruguay', 1117), ('Jamaica', 816), ('Guyana', 350), ('Bahamas', 219), ('Cayman Islands', 203), ('Bermuda', 153), ('Trinidad and Tobago', 141), ('Aruba', 117), ('Barbados', 106), ('Turks and Caicos islands', 86), ('Sint Maarten', 81), ('Antigua and Barbuda', 76), ('Saint Vincent and the Grenadines', 50), ('Belize', 43), ('Curaçao', 28), ('Saint Lucia', 23), ('Grenada', 23), ('Dominica', 18), ('Saint Kitts and Nevis', 17), ('Greenland', 13), ('Falkland Islands (Malvinas)', 13), ('Montserrat', 12), ('Bonaire, Saint Eustatius an

Now I got the result that's more probable.

Debugging is done.

### Investigating the progress of COVID-19 by continent
* Grouping regions by continent
* Month is used as the time measurement unit

In [11]:
covid['YEAR'] = pd.to_datetime(covid['REPORT_DATE']).dt.year
covid['MONTH'] = pd.to_datetime(covid['REPORT_DATE']).dt.month
continent_case = covid[['CONTINENT_NAME', 'PEOPLE_POSITIVE_NEW_CASES_COUNT','YEAR', 'MONTH']].groupby(
                       ['CONTINENT_NAME','YEAR', 'MONTH'], as_index=False).sum()
print(continent_case.loc[continent_case['CONTINENT_NAME'] == 'America'])

   CONTINENT_NAME  YEAR  MONTH  PEOPLE_POSITIVE_NEW_CASES_COUNT
6         America  2020      1                               11
7         America  2020      2                              118
8         America  2020      3                           213560
9         America  2020      4                          1115959
10        America  2020      5                          1542539
11        America  2020      6                          2409763
12        America  2020      7                          2848511


Let's see if the numbers add up correctly

In [12]:
total = 0
for item in countries_continent:
    total += item[1]
print(f'A total of cases from all countries in America: {total}')
print(f"A total of cases from time periods in America: {continent_case.loc[continent_case['CONTINENT_NAME'] == 'America']['PEOPLE_POSITIVE_NEW_CASES_COUNT'].sum()}")

A total of cases from all countries in America: 8130461
A total of cases from time periods in America: 8130461


It does work as intended!

Now, I can start making graphs